# Highrise FAQ Chatbot Development Notebook
This notebook implements a RAG-based chatbot for Highrise FAQ using:
- LangChain
- Google's Generative AI (Gemini)
- MongoDB Atlas Vector Store

## Imports and Setup
Setup environment and import required libraries

In [5]:
import os
import json
import logging
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from pymongo import MongoClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch

# Configure logging
logging.basicConfig(level=logging.INFO)

## Helper Functions

In [6]:
def parse_json_to_documents(json_data):
    """
    Convert JSON FAQ data into Langchain Documents format
    
    Args:
        json_data (dict): Raw JSON data containing FAQ collections
        
    Returns:
        list: List of Langchain Document objects
    """
    documents = []
    if "collections" in json_data:
        for collection in json_data["collections"]:
            for article in collection.get("articles", []):
                content = f"{article.get('question', '')}\n {article.get('answer', '')}\n"
                doc = Document(
                    page_content=content,
                    metadata={"source": article.get("url", "unknown")},
                )
                documents.append(doc)
    return documents

## Vector Store Initialization

In [8]:
def initialize_vector_store(faq_file: str, connection_string: str):
    """
    Initialize MongoDB Atlas Vector Store with FAQ data
    
    Args:
        faq_file (str): Path to FAQ JSON file
        connection_string (str): MongoDB connection URI
        
    Returns:
        MongoDBAtlasVectorSearch: Initialized vector store
    """
    try:
        # Initialize embeddings
        embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
        
        # Load and process documents
        if not os.path.exists(faq_file):
            raise FileNotFoundError(f"FAQ file not found: {faq_file}")
            
        with open(faq_file, "r", encoding="utf-8") as f:
            faq_data = json.load(f)
            
        documents = parse_json_to_documents(faq_data)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=150
        )
        docs = text_splitter.split_documents(documents)
        
        # Configure MongoDB
        client = MongoClient(connection_string)
        DB_NAME = "langchain_faq_db_test"
        COLLECTION_NAME = "langchain_faq_vectorstores_test"
        ATLAS_VECTOR_SEARCH_INDEX_NAME = "langchain-faq-index-vectorstores_test"
        MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]
        
        # Initialize vector store
        vector_store = MongoDBAtlasVectorSearch.from_documents(
            documents=docs,
            collection=MONGODB_COLLECTION,
            embedding=embeddings,
            index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
            relevance_score_fn="cosine",
        )
        vector_store.create_vector_search_index(dimensions=768)
        logging.info(f"Vector store initialized with {len(docs)} documents.")
        return vector_store
        
    except Exception as e:
        logging.error(f"Error during vector store initialization: {e}")
        raise



## Environment Setup & Initialize Vector Store
Set required environment variables

In [ ]:
# Replace with your actual API key
os.environ["GOOGLE_API_KEY"] = "your-api-key"
connection_string = "your mongodb connection string"
faq_file = "faq_data.json"
vector_store = initialize_vector_store(faq_file, connection_string)

In [12]:
def setup_retrieval_chain(vector_store):
    """
    Set up the retrieval-augmented generation chain
    
    Args:
        vector_store: Initialized vector store
    
    Returns:
        Chain: Configured RAG chain
    """
    # Configure retriever
    retriever = vector_store.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": 6}
    )
    
    # Define system prompt
    system_prompt = (
            """
                You are an AI assistant designed to answer questions based on specific context provided from the Highrise FAQ. Your role is to help users by giving clear, friendly, and concise answers using only the retrieved context provided to you. Follow these rules strictly:

                1. Answer Based Only on the Provided Context:
                - If a specific answer to the question is in the context, use that information directly.
                - If the context provides multiple possible answers, prompt the user for clarification by listing the available options.
                - If no relevant information is available, say: "I'm not sure about that, but you can find more information at the Highrise FAQ: https://support.highrise.game/en/".

                2. Be Concise and Friendly:
                - Always end the response with a friendly phrase, such as "Let me know if you need more help!".

                3. Ask for Clarification if Needed:
                - If the user's question is unclear, respond with: "Could you please clarify what you mean? I can assist with topics related to " and also list the available options.
                """
            "\n\n"
            "{context}"
        )
    
    # Create prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])
    
    # Initialize LLM and create chain
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(retriever, question_answer_chain)

## Test the Chatbot

In [13]:
rag_chain = setup_retrieval_chain(vector_store)
test_questions = [
    "How can I delete my account?",
    "What are inbox calls?",
    "How do I report inappropriate behavior?"
]

for question in test_questions:
    print(f"\nQuestion: {question}")
    response = rag_chain.invoke({"input": question})
    print(f"Answer: {response['answer']}")


Question: How can I delete my account?
Answer: To permanently delete your Highrise account, you'll need the latest app version.  Open your profile, tap the gear icon, select 'Account Management', then 'Delete Your Account'.  Enter your Safety Lock (if enabled) and password to begin the process. You have 24 hours to cancel. After that, your account will be deactivated, and all data will be permanently deleted after 30 days.  Remember, deleting your account is irreversible!

Let me know if you need more help!


Question: What are inbox calls?
Answer: Inbox Calls is a feature that lets you voice chat directly within Highrise, no matter where you are in the app.  You can make private 1:1 calls, join group calls, or participate in crew voice chats. Let me know if you need more help!


Question: How do I report inappropriate behavior?
Answer: To report inappropriate behavior in Highrise, you can use the in-app reporting tool.  The process varies slightly depending on what you're reporting (